### Extracting information from Yellow Pages website

In [1]:
import requests
import time
import random
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from IPython.core.display import display, HTML
import openpyn

C:\Users\Aldie\AppData\Local\Temp\ipykernel_250996\1418911603.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [236]:
electrical-equipment-appliance-and-component

SyntaxError: invalid syntax (1040427280.py, line 1)

In [3]:
DF_Electrical = pd.DataFrame(columns = ['Category', 'Subcategory','Trade Name', 'Business Name', 'Location', 'Rating', 'Review Link', 'Phone', 'Landline', 'Socials', 'Email', 'Link'])

In [5]:
def extractor(url, category, maindf):
    #Consider first the first page of the website
    main_url = url
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    #Open the website using BS4 and get the main business sites
    MAIN_response = requests.get(url, headers = headers)
    MAIN_soup = BeautifulSoup(MAIN_response.content, 'html.parser')

    #Extract all URLs of businesses under the first page
    LIST_biz = MAIN_soup.find_all('a', 'search-business-img yp-click')
    pageX_biz = []
    for i in range(len(LIST_biz)):
        url = f'https://yellow-pages.ph' + LIST_biz[i].get('href')
        pageX_biz.append(url)

    #Manually open them
    for URL_biz in pageX_biz:
        URL_response = requests.get(URL_biz, headers=headers)
        soup = BeautifulSoup(URL_response.content, 'html.parser')

        #Temporary container
        list = []
        
        #Get all relevant information
        CATEGORY = category
        SUBCATEGORY = (soup.find('a', 'capsule-rounded').text.strip() if soup.find('a', 'capsule-rounded') else 'N/A')
        TRADENAME = (soup.find('h1', 'h1-tradename').text.strip() if soup.find('h1', 'h1-tradename') else 'N/A')
        BIZNAME = (
        soup.find('h2', 'h2-businessname').text.strip() 
        if soup.find('h1', 'h1-tradename') 
        else soup.find('h1', 'h1-single-businessname').text.strip() 
        if soup.find('h1', 'h1-single-businessname') 
        else 'N/A'  # or some default value if neither is found
)
        LOCATION = (soup.find('a', 'biz-link yp-click').text.strip() if soup.find('a', 'biz-link yp-click') else 'N/A')
        RATING = (soup.find('div', 'search-rating-empty search-star-number').text.strip() if soup.find('div', 'search-rating-empty search-star-number') else 'N/A')
        REVIEW = (f'https://yellow-pages.ph' + soup.find('a', href=lambda href: href and 'reviews' in href).get('href') if soup.find('a', href=lambda href: href and 'reviews' in href) else 'N/A')
        PHONE = (soup.find('a', 'mr-1 biz-btn-call btn btn-yp-default mb-2 px-xl-4 yp-click min-width-0 min-width-100-btn').get('data-phone') if soup.find('a', 'mr-1 biz-btn-call btn btn-yp-default mb-2 px-xl-4 yp-click min-width-0 min-width-100-btn') else 'N/A')
        LANDLINE = (soup.find('a', 'mr-1 biz-btn-call btn btn-yp-default mb-2 px-xl-4 yp-click min-width-0 min-width-100-btn').get('href') if soup.find('a', 'mr-1 biz-btn-call btn btn-yp-default mb-2 px-xl-4 yp-click min-width-0 min-width-100-btn') else 'N/A')
        SOCIALS = (soup.find('a', 'btn biz-link mr-2 padding-2 d-block ellipsis yp-click social-media-link text-light messenger-bg').get('href') if soup.find('a', 'btn biz-link mr-2 padding-2 d-block ellipsis yp-click social-media-link text-light messenger-bg') else 'N/A')
        EMAIL = (soup.find('a','biz-link d-block ellipsis yp-click email-link').get('href') if soup.find('a','biz-link d-block ellipsis yp-click email-link') else 'N/A')
        MAINLINK = URL_biz
        
        #Append all entries
        list.append(CATEGORY)
        list.append(SUBCATEGORY)
        list.append(TRADENAME)
        list.append(BIZNAME)
        list.append(LOCATION)
        list.append(RATING)
        list.append(REVIEW)
        list.append(PHONE)
        list.append(LANDLINE)
        list.append(SOCIALS)
        list.append(EMAIL)
        list.append(MAINLINK)

        #Append them
        list = [list]
        print(list)
        TEMP_df = pd.DataFrame(list, columns = maindf.columns)
        maindf = pd.concat([TEMP_df, maindf], ignore_index=True)
        print(f'Done with {URL_biz}')
        
        #Interval to PREVENT IP ADDRESS BLOCKING
        time.sleep(random.randint(7,10))

    #Print test to know our current page
    print(f'Done with {main_url}')

    #Interval to PREVENT IP ADDRESS BLOCKING
    time.sleep(random.randint(7, 10))
   
    #Go to the next page by doing recursion
    if MAIN_soup.find_all('a', {'page-link', 'next'}):
        new_url = f'https://yellow-pages.ph' + MAIN_soup.find_all('a', {'page-link', 'next'})[-1].get('href')
        maindf = extractor(new_url, category, maindf)

    return maindf

In [7]:
DF_Electrical = pd.DataFrame(columns = ['Category', 'Subcategory','Trade Name', 'Business Name', 'Location', 'Rating', 'Review Link', 'Phone', 'Landline', 'Socials', 'Email', 'Link'])
DF_Electrical = extractor('https://www.yellow-pages.ph/category/electrical-equipment-appliance-and-component/page-1', 'DF_Electrical', DF_Electrical)
DF_Electrical.to_csv('DF_Electrical.csv') 

[['DF_Electrical', 'Water Treatment Equipment and Services', 'N/A', 'BGR HYDROLAB TESTING CENTER', 'Mindanao Avenue , Quezon City 1116 Metro Manila', 'No Ratings', 'https://yellow-pages.ph/business/bgr-hydrolab-testing-center-4/reviews', '+63(2)84041065', 'tel:+63284041065', 'N/A', 'N/A', 'https://yellow-pages.ph/business/bgr-hydrolab-testing-center-4']]
Done with https://yellow-pages.ph/business/bgr-hydrolab-testing-center-4
[['DF_Electrical', 'Lighting', 'N/A', '1 Audio Lights And Sounds  Corporation', '23 Colt Street, Rancho Estate 1, Concepcion Dos, Marikina City 1800 Metro Manila', 'No Ratings', 'https://yellow-pages.ph/business/1-audio-lights-and-sounds-corporation/reviews', '+63(2)86620408', 'tel:+63286620408', 'N/A', 'mailto:bongalimario@yahoo.com', 'https://yellow-pages.ph/business/1-audio-lights-and-sounds-corporation']]
Done with https://yellow-pages.ph/business/1-audio-lights-and-sounds-corporation
[['DF_Electrical', 'Lighting Fixtures', '1-Adler International Enterprises',

In [300]:
Catering = pd.DataFrame(columns = ['Category', 'Subcategory','Trade Name', 'Business Name', 'Location', 'Rating', 'Review Link', 'Phone', 'Landline', 'Socials', 'Email', 'Link'])
Catering = extractor('https://www.yellow-pages.ph/category/catering/page-1', 'Catering', Catering)
Catering.to_csv('Catering.csv')

[['Catering', 'Chairs and Tables - Renting', 'N/A', '1 Jems Chairs & Tables Sales & Rental', '914 Fabella Road, Mandaluyong City, Metro Manila', 'No Ratings', 'https://yellow-pages.ph/business/1-jems-chairs-tables-sales-rental/reviews', '+63(2)5322556', 'tel:+6325322556', 'N/A', 'N/A', 'https://yellow-pages.ph/business/1-jems-chairs-tables-sales-rental']]
Done with https://yellow-pages.ph/business/1-jems-chairs-tables-sales-rental
[['Catering', 'Catering', 'N/A', '128 Catering', '67 Epifanio de los Santos Avenue, Cubao, Quezon City, Metro Manila', 'No Ratings', 'https://yellow-pages.ph/business/128-catering/reviews', '+63(917)6261697', 'tel:+639176261697', 'N/A', 'mailto:128catering@gmail.com', 'https://yellow-pages.ph/business/128-catering']]
Done with https://yellow-pages.ph/business/128-catering
[['Catering', 'Catering', 'N/A', '128 Catering Services', '65 8th Avenue, corner Main Avenue, Quezon City, Metro Manila', 'No Ratings', 'https://yellow-pages.ph/business/128-catering-service

In [9]:
Photography = pd.DataFrame(columns = ['Category', 'Subcategory','Trade Name', 'Business Name', 'Location', 'Rating', 'Review Link', 'Phone', 'Landline', 'Socials', 'Email', 'Link'])
Photography = extractor('https://www.yellow-pages.ph/category/photography-services/page-1', 'Photography', Photography)
Photography.to_csv('Photography.csv')

[['Photography', 'Photographic Services', 'N/A', 'Federation of Philippine Photographers Foundation', '302-Annex FEMII Building, A. Soriano Street,  Intramuros, City of Manila, Metro Manila', 'No Ratings', 'https://yellow-pages.ph/business/federation-of-philippine-photographers-foundation/reviews', '+63(2)85247576', 'tel:+63285247576', 'N/A', 'mailto:myfppf@yahoo.com', 'https://yellow-pages.ph/business/federation-of-philippine-photographers-foundation']]
Done with https://yellow-pages.ph/business/federation-of-philippine-photographers-foundation
[['Photography', 'Photography Services', 'N/A', '20 20 Photo Center', 'Chipeco Av, Calamba City 4027 Laguna', 'No Ratings', 'https://yellow-pages.ph/business/20-20-photo-center/reviews', '+63(49)5454265', 'tel:+63495454265', 'N/A', 'N/A', 'https://yellow-pages.ph/business/20-20-photo-center']]
Done with https://yellow-pages.ph/business/20-20-photo-center
[['Photography', 'Photography Services', 'N/A', '22nd Story Studio', '25 Samar Avenue, Sout

In [11]:
Accounting = pd.DataFrame(columns = ['Category', 'Subcategory','Trade Name', 'Business Name', 'Location', 'Rating', 'Review Link', 'Phone', 'Landline', 'Socials', 'Email', 'Link'])
Accounting = extractor('https://www.yellow-pages.ph/category/accounting-services/page-1', 'Accounting', Accounting)
Accounting.to_csv('Accounting.csv')

[['Accounting', 'Accounting Services', 'N/A', '101 Solutions Accounting Firm', 'Rivera Building, Quezon Ave, Catbangen, San Fernando City 2500 La Union', 'N/A', 'https://yellow-pages.ph/business/101-solutions-accounting-firm/reviews', '+63(72)6074972', 'tel:+63726074972', 'N/A', 'N/A', 'https://yellow-pages.ph/business/101-solutions-accounting-firm']]
Done with https://yellow-pages.ph/business/101-solutions-accounting-firm
[['Accounting', 'Accountants', 'N/A', 'A D Pascual & Associates', 'F Ta¤Edo, Tarlac City, Tarlac', 'No Ratings', 'https://yellow-pages.ph/business/a-d-pascual-associates/reviews', '+63(45)9821771', 'tel:+63459821771', 'N/A', 'N/A', 'https://yellow-pages.ph/business/a-d-pascual-associates']]
Done with https://yellow-pages.ph/business/a-d-pascual-associates
[['Accounting', 'Accounting Services', 'N/A', 'A V Tidoso And Company', 'M.C. Briones Street, Mandaue City, Cebu', 'No Ratings', 'https://yellow-pages.ph/business/a-v-tidoso-company/reviews', '+63(32)3440344', 'tel:

In [ ]:
HealthCare = pd.DataFrame(columns = ['Category', 'Subcategory','Trade Name', 'Business Name', 'Location', 'Rating', 'Review Link', 'Phone', 'Landline', 'Socials', 'Email', 'Link'])
HealthCare = extractor('https://www.yellow-pages.ph/category/health-care-services/page-1', 'HealthCare', HealthCare)
HealthCare.to_csv('HealthCare.csv')

[['HealthCare', 'Diagnostic Laboratory', 'N/A', 'Arguelles Medical Clinic', '1271 Zobel Roxas Avenue, Malate, City of Manila 1004 Metro Manila', 'No Ratings', 'https://yellow-pages.ph/business/arguelles-medical-clinic-2/reviews', '+63(2)85215353', 'tel:+63285215353', 'N/A', 'mailto:email@a-md.com', 'https://yellow-pages.ph/business/arguelles-medical-clinic-2']]
Done with https://yellow-pages.ph/business/arguelles-medical-clinic-2
[['HealthCare', 'Blood Banks', 'N/A', '(Rp) Phil National Red Cross', 'Pettit Barracks Road, Zamboanga City, Zamboanga Del Sur', 'No Ratings', 'https://yellow-pages.ph/business/rp-phil-national-red-cross/reviews', '+63(62)9912015', 'tel:+63629912015', 'N/A', 'N/A', 'https://yellow-pages.ph/business/rp-phil-national-red-cross']]
Done with https://yellow-pages.ph/business/rp-phil-national-red-cross
[['HealthCare', 'Medical Cosmetics', 'N/A', "2Ra's Pharmacy", '1057 Hermosa Street, Tondo, City of Manila 1012 Metro Manila', 'No Ratings', 'https://yellow-pages.ph/b

In [145]:
print(1)

1


In [191]:
# Locate the <a> tag with the class 'yp-click'
element = driver.find_element("xpath","//a[@class='yp-click']")

print(element.text)

In [191]:
# Locate the <a> tag with the class 'yp-click'
element = driver.find_element("xpath","//a[@class='yp-click']")

print(element.text)

In [521]:
url2 = 'https://www.yellow-pages.ph/category/electrical-equipment-appliance-and-component/page-1'

driver = webdriver.Chrome()
driver.get(url2)

In [213]:
element = driver.find_elements("xpath","//a[@class='yp-click']")

for el in element:
    # Extract details
    text = el.text  # Get the visible text inside the <a> tag
    href = el.get_attribute("href")  # Get the 'href' attribute (URL)
    data_log = el.get_attribute("data-log")  # Get the 'data-log' attribute
    data_permalink = el.get_attribute("data-permalink")  # Get the 'data-permalink' attribute
    data_module = el.get_attribute("data-module")  # Get the 'data-module' attribute
    data_section = el.get_attribute("data-section")  # Get the 'data-section' attribute
    data_type = el.get_attribute("data-type")  # Get the 'data-type' attribute

    print(href)

https://s-tank.ph/
https://www.yellow-pages.ph/yp_cookies


In [209]:
data_log

'209'

In [177]:
element = driver.find_elements("xpath", "//*[contains(@class, 'search-business-otherinfoearch-business-name')]")
for x in element:
    print(x.text)


In [154]:
element = driver.find_elements("xpath", "//*[contains(@class, 'search-capsule-rounded search-badge')]")
print(len(element))
for x in element:
    print(x.text)

15
Water Treatment Equipment and Services
Lighting
Lighting Fixtures
Lighting
Air Conditioning Equipment and Systems
Electric Equipment - Service and Repairing
Air Conditioning Equipment and Systems
Air Conditioning Equipment and Systems
Electric Equipment - Service and Repairing
Electric Equipment - Service and Repairing
Electric Supplies
Air Conditioning Equipment and Systems
Electric Equipment - Service and Repairing
Video Cassette Tapes-Sales and Rental
Electric Transformers


In [152]:
element = driver.find_elements("xpath", "//*[contains(@class, 'search-rating-empty mt-1')]")
print(len(element))
for x in element:
    print(x.text)

15
No Ratings
No Ratings
No Ratings
No Ratings
No Ratings
No Ratings
No Ratings
No Ratings
No Ratings
No Ratings
No Ratings
No Ratings
No Ratings
No Ratings
No Ratings


In [166]:
element = driver.find_elements("xpath", "//*[contains(@class, 'ellipsis')]")
print(len(element))
for x in element:
    print(x.text)

101
Mindanao Avenue , Quezon City 1116 Metro Manila
23 Colt Street, Rancho Estate 1, Concepcion Dos, Marikina City 1800 Metro Manila
572 Tolentino Street, Pasay City 1300 Metro Manila
1089 Primero De Mayo Street, Tondo, City of Manila 1000 Metro Manila
54 National Highway, Visayan Village, Purok Aquarius, Visayan Village, Tagum City, Davao Del Norte
Block 01 Lot 06, Phase 1 2nd Street, Balubad Settlement, Barangay Nangka, Marikina City 1820 Metro Manila
J Aguilar Avenue, Las Pinas City, Metro Manila
Suite 1003 The Infinity, 26th Street, Bonifacio Global City, Taguig City 1632 Metro Manila
B57 L5 CAA Road, Phase 3, Las Pinas City, Metro Manila
51-B Agno Street, Banawe, Quezon City 1113 Metro Manila
Provl Rd, Cabatuan 3315 Isabela
14A Gold Tree Building, Sta. Rita Corner Canal Roads, SBFZ, Olongapo City 2222 Zambales
9580 Kalayaan Avenue, Guadalupe, Makati City, Metro Manila
Angsco Mansion, Caloocan City 1400 Metro Manila
641 Lenoria Street, Paco, City of Manila 1007 Metro Manila









In [ ]:

from selenium import webdriver

# Initialize the WebDriver
driver = webdriver.Chrome()

# Open the page (replace with the actual URL)
driver.get('https://example.com')

# 1. Find the <h2> element with a specific class (e.g., 'section-title')
h2_element = driver.find_element_by_xpath("//h2[@class='section-title']")

# 2. Find all relevant elements that are beneath this <h2> tag
# Option 1: Find all child elements within the <h2> section (like paragraphs, divs, etc.)
relevant_elements = h2_element.find_elements_by_xpath("following-sibling::*")

# 3. Loop through the relevant elements and extract their content
for element in relevant_elements:
    # Extract text or attributes from each element
    print("Element Text:", element.text)

# Option 2: If you know specific elements (e.g., <p>, <div>, <a>) under the <h2> tag, 
# you can refine the XPath to target only those tags.
paragraphs = h2_element.find_elements_by_xpath("following-sibling::p")
for paragraph in paragraphs:
    print("Paragraph Text:", paragraph.text)

# Example: Extract all links (anchors) under the <h2> section
links = h2_element.find_elements_by_xpath("following-sibling::a")
for link in links:
    print("Link Text:", link.text, "URL:", link.get_attribute('href'))

# Example: Extract all divs under the <h2> section
divs = h2_element.find_elements_by_xpath("following-sibling::div")
for div in divs:
    print("Div Text:", div.text)

# Close the driver once done
driver.quit()


In [24]:
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [263]:
test = soup.find_all('a', 'search-business-img yp-click')

for a in test:
    url = f'https://yellow-pages.ph' + a.get('href')
    #requests.get(url)
    print(url)
    #print("Success")

https://yellow-pages.ph/business/bgr-hydrolab-testing-center-4
https://yellow-pages.ph/business/1-audio-lights-and-sounds-corporation
https://yellow-pages.ph/business/1-adler-international-enterprises-incorporated
https://yellow-pages.ph/business/10-inch-lights-and-sound
https://yellow-pages.ph/business/101-repair-shop
https://yellow-pages.ph/business/102-trading-2
https://yellow-pages.ph/business/1rotary-trading-corporation-13
https://yellow-pages.ph/business/1rotary-trading-corporation-10
https://yellow-pages.ph/business/1st-corinthians-appliance-services
https://yellow-pages.ph/business/215-luxury-car-aircon-services
https://yellow-pages.ph/business/3h-electrical-supply
https://yellow-pages.ph/business/3k-aircool-incorporated-2
https://yellow-pages.ph/business/620-enterprises
https://yellow-pages.ph/business/6th-av-video-resources
https://yellow-pages.ph/business/7-j-verdejo-enterprises


In [269]:
tre2 = requests.get("https://yellow-pages.ph/business/bgr-hydrolab-testing-center-4")
soup = BeautifulSoup(tre2.content, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en-PH">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"0238f0b341","applicationID":"94636524","transactionName":"IQxXERNbDQoGQxxKRT0BTBYIWgQVEB5aXVEHGw==","queueTime":0,"applicationTime":138,"agent":""}</script>
<script type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={xpid:"XQIAVFZVGwoEUlJUAgMD",licenseKey:"0238f0b341",applicationID:"94636524"};;/*! For license information please see nr-loader-spa-1.281.0.min.js.LICENSE.txt */
(()=>{var e,t,r={8122:(e,t,r)=>{"use strict";r.d(t,{a:()=>i});var n=r(944);function i(e,t){try{if(!e||"object"!=typeof e)return(0,n.R)(3);if(!t||"object"!=typeof t)return(0,n.R)(4);const r=Object.create(Object.getPrototypeOf(t),Object.getOwnPropertyDescriptors(t)),o=0===Ob

In [345]:
soup.find('a', href=lambda href: href and 'reviews' in href).get('href')

'/business/bgr-hydrolab-testing-center-4/reviews'

In [225]:
soup

<!DOCTYPE html>

<html lang="en-PH">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"0238f0b341","applicationID":"94636524","transactionName":"IQxXERNbDQoGQxxQVBYGXgoTTU4FAkVWVFoQGmYHFEcICAZCQFZG","queueTime":0,"applicationTime":2719,"agent":""}</script>
<script type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={xpid:"XQIAVFZVGwoEUlJUAgMD",licenseKey:"0238f0b341",applicationID:"94636524"};;/*! For license information please see nr-loader-spa-1.281.0.min.js.LICENSE.txt */
(()=>{var e,t,r={8122:(e,t,r)=>{"use strict";r.d(t,{a:()=>i});var n=r(944);function i(e,t){try{if(!e||"object"!=typeof e)return(0,n.R)(3);if(!t||"object"!=typeof t)return(0,n.R)(4);const r=Object.create(Object.getPrototypeOf(t),Object.getOwnPropertyDescriptors

In [227]:
soup.find_all('span', 'search-capsule-rounded search-badge')

[<span class="search-capsule-rounded search-badge">Water Treatment Equipment and Services</span>,
 <span class="search-capsule-rounded search-badge">Lighting</span>,
 <span class="search-capsule-rounded search-badge">Lighting Fixtures</span>,
 <span class="search-capsule-rounded search-badge">Lighting</span>,
 <span class="search-capsule-rounded search-badge">Air Conditioning Equipment and Systems</span>,
 <span class="search-capsule-rounded search-badge">Electric Equipment - Service and Repairing</span>,
 <span class="search-capsule-rounded search-badge">Air Conditioning Equipment and Systems</span>,
 <span class="search-capsule-rounded search-badge">Air Conditioning Equipment and Systems</span>,
 <span class="search-capsule-rounded search-badge">Electric Equipment - Service and Repairing</span>,
 <span class="search-capsule-rounded search-badge">Electric Equipment - Service and Repairing</span>,
 <span class="search-capsule-rounded search-badge">Electric Supplies</span>,
 <span class

In [229]:
soup.find_all('span', 'ellipsis')

[<span class="ellipsis">Mindanao Avenue , Quezon City 1116 Metro Manila </span>,
 <span class="ellipsis">23 Colt Street, Rancho Estate 1, Concepcion Dos, Marikina City 1800 Metro Manila </span>,
 <span class="ellipsis">572 Tolentino Street, Pasay City 1300 Metro Manila </span>,
 <span class="ellipsis">1089 Primero De Mayo Street, Tondo, City of Manila 1000 Metro Manila </span>,
 <span class="ellipsis">54 National Highway, Visayan Village, Purok Aquarius, Visayan Village, Tagum City, Davao Del Norte </span>,
 <span class="ellipsis">Block 01 Lot 06, Phase 1 2nd Street, Balubad Settlement, Barangay Nangka, Marikina City 1820 Metro Manila </span>,
 <span class="ellipsis">J Aguilar Avenue, Las Pinas City, Metro Manila </span>,
 <span class="ellipsis">Suite 1003 The Infinity, 26th Street, Bonifacio Global City, Taguig City 1632 Metro Manila </span>,
 <span class="ellipsis">B57 L5 CAA Road, Phase 3, Las Pinas City, Metro Manila </span>,
 <span class="ellipsis">51-B Agno Street, Banawe, Quezon

In [233]:
soup.find_all('div', 'search-rating-empty mt-1')

[<div class="search-rating-empty mt-1">
 <i class="far fa-star"></i> No Ratings
                       </div>,
 <div class="search-rating-empty mt-1">
 <i class="far fa-star"></i> No Ratings
                       </div>,
 <div class="search-rating-empty mt-1">
 <i class="far fa-star"></i> No Ratings
                       </div>,
 <div class="search-rating-empty mt-1">
 <i class="far fa-star"></i> No Ratings
                       </div>,
 <div class="search-rating-empty mt-1">
 <i class="far fa-star"></i> No Ratings
                       </div>,
 <div class="search-rating-empty mt-1">
 <i class="far fa-star"></i> No Ratings
                       </div>,
 <div class="search-rating-empty mt-1">
 <i class="far fa-star"></i> No Ratings
                       </div>,
 <div class="search-rating-empty mt-1">
 <i class="far fa-star"></i> No Ratings
                       </div>,
 <div class="search-rating-empty mt-1">
 <i class="far fa-star"></i> No Ratings
                       </div>,
 

In [6]:
# Base URL for the Yellow Pages Philippines directory
base_url = 'https://www.yellow-pages.ph/business-directory'

# List to store the scraped business data
businesses = []

response = requests.get(base_url)
soup = BeautifulSoup(response.content, 'html.parser')

In [8]:
soup

<!DOCTYPE html>

<html lang="en-PH">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport"/>
<link href="https://static1.eyellowpages.ph/assets/favicon-21c6574a65787587fe8b1d3136ca98a63ea745a7b31dc5a559a14641fe0f6b7d.ico" rel="shortcut icon"/>
<link href="https://static1.eyellowpages.ph/assets/favicon-21c6574a65787587fe8b1d3136ca98a63ea745a7b31dc5a559a14641fe0f6b7d.ico" rel="icon"/>
<title>Error - 404 (Not Found) | Yellow Pages PH</title>
<meta content="" name="user-id">
<!-- HTML5 shim and Respond.js for IE8 support of HTML5 elements and media queries -->
<!--[if lt IE 9]>
  <script src="https://oss.maxcdn.com/html5shiv/3.7.2/html5shiv.min.js"></script>
  <script src="https://oss.maxcdn.com/respond/1.4.2/respond.min.js"></script>
  <![endif]-->
<meta content="#fbd13f" name="theme-color"/>
<meta content="#fbd13f" name="msapplication-navbutton-color"/>
<

In [ ]:

# Function to scrape businesses from a given URL
def scrape_businesses(page_url):
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find business listings (adjust the tag/class based on actual HTML structure)
    business_list = soup.find_all('div', class_='listing')  # Adjust class as per website structure
    
    for business in business_list:
        # Extracting business name and contact information (adjust based on structure)
        name = business.find('h2').text.strip() if business.find('h2') else 'N/A'
        contact = business.find('span', class_='contact').text.strip() if business.find('span', class_='contact') else 'N/A'
        
        businesses.append({'Business Name': name, 'Contact Info': contact})

# Function to get total pages for pagination (if applicable)
def get_total_pages(soup):
    pagination = soup.find('div', class_='pagination')  # Adjust based on pagination structure
    total_pages = 1  # Default if no pagination is found
    if pagination:
        total_pages = int(pagination.find_all('a')[-2].text.strip())  # Last page number
    return total_pages

# Scrape the first page and find total pages
response = requests.get(base_url)
soup = BeautifulSoup(response.content, 'html.parser')
total_pages = get_total_pages(soup)

# Scraping multiple pages if pagination exists
for page in range(1, total_pages + 1):
    next_page_url = f'{base_url}?page={page}'
    scrape_businesses(next_page_url)

# Convert the list to a DataFrame and save as CSV
df = pd.DataFrame(businesses)
df.to_csv('yellow_pages_businesses.csv', index=False)
print("Data scraped and saved to yellow_pages_businesses.csv")